In [34]:
import numpy as np
import tensorflow as tf
tf.set_random_seed(777)
sess = tf.Session()

In [58]:
xy = np.loadtxt('test-score.csv', delimiter=',', dtype=np.float32)

a = np.array([1,2,3,4,5])

a[-2]

In [70]:
x_data = xy[:,:-1]
y_data = xy[:,[-1]]

In [74]:
print(x_data.shape, x_data, len(x_data))
print(y_data.shape, y_data)

((25, 3), array([[  73.,   80.,   75.],
       [  93.,   88.,   93.],
       [  89.,   91.,   90.],
       [  96.,   98.,  100.],
       [  73.,   66.,   70.],
       [  53.,   46.,   55.],
       [  69.,   74.,   77.],
       [  47.,   56.,   60.],
       [  87.,   79.,   90.],
       [  79.,   70.,   88.],
       [  69.,   70.,   73.],
       [  70.,   65.,   74.],
       [  93.,   95.,   91.],
       [  79.,   80.,   73.],
       [  70.,   73.,   78.],
       [  93.,   89.,   96.],
       [  78.,   75.,   68.],
       [  81.,   90.,   93.],
       [  88.,   92.,   86.],
       [  78.,   83.,   77.],
       [  82.,   86.,   90.],
       [  86.,   82.,   89.],
       [  78.,   83.,   85.],
       [  76.,   83.,   71.],
       [  96.,   93.,   95.]], dtype=float32), 25)
((25, 1), array([[ 152.],
       [ 185.],
       [ 180.],
       [ 196.],
       [ 142.],
       [ 101.],
       [ 149.],
       [ 115.],
       [ 175.],
       [ 164.],
       [ 141.],
       [ 141.],
       [ 184.],
 

In [42]:
print(x_data)

[[  73.   80.   75.]
 [  93.   88.   93.]
 [  89.   91.   90.]
 [  96.   98.  100.]
 [  73.   66.   70.]
 [  53.   46.   55.]
 [  69.   74.   77.]
 [  47.   56.   60.]
 [  87.   79.   90.]
 [  79.   70.   88.]
 [  69.   70.   73.]
 [  70.   65.   74.]
 [  93.   95.   91.]
 [  79.   80.   73.]
 [  70.   73.   78.]
 [  93.   89.   96.]
 [  78.   75.   68.]
 [  81.   90.   93.]
 [  88.   92.   86.]
 [  78.   83.   77.]
 [  82.   86.   90.]
 [  86.   82.   89.]
 [  78.   83.   85.]
 [  76.   83.   71.]
 [  96.   93.   95.]]


In [75]:
X = tf.placeholder(tf.float32, shape=[None,3])
Y = tf.placeholder(tf.float32, shape=[None,1])

W = tf.Variable(tf.random_normal([3,1]), name = 'weight')
b = tf.Variable(tf.random_normal([1]), name = 'bias')

In [47]:
print(sess.run(tf.random_normal([3,1])))

[[ 1.24536574]
 [-1.78801703]
 [-0.75885421]]


In [76]:
hypo = tf.matmul(X,W) + b
cost = tf.reduce_mean(tf.square(hypo - Y))

In [77]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

In [79]:
sess.run(tf.global_variables_initializer())

for step in range(2001):
    cost_val, hy_val, _ = sess.run([cost, hypo, train],
                                  feed_dict={X:x_data, Y:y_data})
    if step % 10 == 0:
        print(step, " ", cost_val, " ", hy_val)

(0, ' ', 4294.7383, ' ', array([[  85.38829041],
       [ 118.3828125 ],
       [ 108.68049622],
       [ 117.90905762],
       [  94.31590271],
       [  70.16174316],
       [  82.76469421],
       [  53.32986069],
       [ 113.46897125],
       [ 105.15555573],
       [  84.89611053],
       [  90.32015228],
       [ 113.18516541],
       [  95.75011444],
       [  85.4050293 ],
       [ 118.17611694],
       [  96.63304901],
       [  95.61501312],
       [ 105.55091858],
       [  92.53330231],
       [  99.69082642],
       [ 109.43678284],
       [  93.83792114],
       [  87.99217224],
       [ 120.57331848]], dtype=float32))
(10, ' ', 59.39526, ' ', array([[ 146.17224121],
       [ 191.40150452],
       [ 180.64622498],
       [ 196.28947449],
       [ 149.99157715],
       [ 111.21128845],
       [ 141.45220947],
       [  96.86214447],
       [ 181.70652771],
       [ 168.3661499 ],
       [ 141.42277527],
       [ 146.04148865],
       [ 187.53065491],
       [ 157.54426575

In [84]:
print('your score will be', sess.run(hypo, feed_dict={X:[[100,70,101]]}))
print('other score will be', sess.run(hypo, feed_dict={X:[[100,70,101],[1,2,3]]}))

('your score will be', array([[ 205.00793457]], dtype=float32))
('other score will be', array([[ 205.00793457],
       [   2.20388937]], dtype=float32))


In [89]:
# file이 너무 커서 queue에 쌓아두고 reader -> decoder 후 최종 queue에 쌓아서 train에 필요한 소량 부분만 읽는 방법

# mulitple files -> filename_queue -> reader/decoder -> example queue
filename_queue = tf.train.string_input_producer(
                    ['test-score.csv', 'test-score_02.csv','test-score_03.csv'],
                    shuffle = False, name = 'filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue) # read

record_defaults = [[0.],[0.],[0.],[0.]] # null값 처리, float 타입

xy = tf.decode_csv(value, record_defaults=record_defaults) # decode

In [90]:
print(xy)

[<tf.Tensor 'DecodeCSV:0' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:1' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:2' shape=() dtype=float32>, <tf.Tensor 'DecodeCSV:3' shape=() dtype=float32>]


In [ ]:
train_x_batch, train_y_batch = \
    tf.train.batch(xy[0:-1], xy[-1:], batch_size = 10)
sess = tf.Session()

In [91]:
# Lab 4 Multi-variable linear regression
# https://www.tensorflow.org/programmers_guide/reading_data

import tensorflow as tf
tf.set_random_seed(777)  # for reproducibility

filename_queue = tf.train.string_input_producer(
    ['test-score.csv'], shuffle=False, name='filename_queue')

reader = tf.TextLineReader()
key, value = reader.read(filename_queue)

# Default values, in case of empty columns. Also specifies the type of the
# decoded result.
record_defaults = [[0.], [0.], [0.], [0.]]
xy = tf.decode_csv(value, record_defaults=record_defaults)

# collect batches of csv in
train_x_batch, train_y_batch = \
    tf.train.batch([xy[0:-1], xy[-1:]], batch_size=10)

# placeholders for a tensor that will be always fed.
X = tf.placeholder(tf.float32, shape=[None, 3])
Y = tf.placeholder(tf.float32, shape=[None, 1])

W = tf.Variable(tf.random_normal([3, 1]), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# Hypothesis
hypothesis = tf.matmul(X, W) + b

# Simplified cost/loss function
cost = tf.reduce_mean(tf.square(hypothesis - Y))

# Minimize
optimizer = tf.train.GradientDescentOptimizer(learning_rate=1e-5)
train = optimizer.minimize(cost)

# Launch the graph in a session.
sess = tf.Session()
# Initializes global variables in the graph.
sess.run(tf.global_variables_initializer())

# Start populating the filename queue.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

for step in range(2001):
    x_batch, y_batch = sess.run([train_x_batch, train_y_batch])
    cost_val, hy_val, _ = sess.run(
        [cost, hypothesis, train], feed_dict={X: x_batch, Y: y_batch})
    if step % 10 == 0:
        print(step, "Cost: ", cost_val, "\nPrediction:\n", hy_val)

coord.request_stop()
coord.join(threads)

# Ask my score
print("Your score will be ",
      sess.run(hypothesis, feed_dict={X: [[100, 70, 101]]}))

print("Other scores will be ",
      sess.run(hypothesis, feed_dict={X: [[60, 70, 110], [90, 100, 80]]}))


(0, 'Cost: ', 59249.887, '\nPrediction:\n', array([[ -88.7726593 ],
       [ -94.68820953],
       [ -99.43746948],
       [-109.03936768],
       [ -68.58809662],
       [ -50.2598114 ],
       [ -86.0574646 ],
       [ -70.36940002],
       [ -86.5897522 ],
       [ -80.74008179]], dtype=float32))
(10, 'Cost: ', 47.372673, '\nPrediction:\n', array([[ 147.39460754],
       [ 189.18133545],
       [ 180.25317383],
       [ 195.57969666],
       [ 147.90112305],
       [ 109.37372589],
       [ 141.97732544],
       [  98.70462799],
       [ 178.72880554],
       [ 165.04360962]], dtype=float32))
(20, 'Cost: ', 49.226196, '\nPrediction:\n', array([[ 148.99864197],
       [ 191.08222961],
       [ 182.13963318],
       [ 197.63960266],
       [ 149.3396759 ],
       [ 110.43952942],
       [ 143.53489685],
       [  99.88200378],
       [ 180.50569153],
       [ 166.69900513]], dtype=float32))
(30, 'Cost: ', 49.032982, '\nPrediction:\n', array([[ 149.01976013],
       [ 191.08036804],
  